#My understanding is the 4 checkpoitns given are the same except for the linear adapter layer. This is my validation that this is true.

In [1]:
print("test2")
import torch
import torch.nn.functional as F
import deepspeed

test2
[2025-06-17 17:47:05,224] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-06-17 17:47:05,228] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/users/9/lee02328/.conda/envs/dynadiff5/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/users/9/lee02328/.conda/envs/dynadiff5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-17 17:47:22.907326: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-17 17:47:26.330559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
if torch.cuda.is_available():
    print(f"GPU is available! Device: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available.")

GPU not available.


In [3]:
def load_and_prep_checkpoint(checkpoint_dir):
    """Loads a DeepSpeed checkpoint and removes the 'model.' prefix."""
    print(f"Loading checkpoint from: {checkpoint_dir}")
    try:
        state_dict = deepspeed.utils.zero_to_fp32.get_fp32_state_dict_from_zero_checkpoint(
            checkpoint_dir=checkpoint_dir,
            tag="checkpoint",
        )
        # Strip the "model." prefix
        state_dict = {
            k[len("model.") :]: v
            for k, v in state_dict.items()
            if k.startswith("model.")
        }
        print(f"-> Successfully loaded {len(state_dict)} layers.")
        return state_dict
    except Exception as e:
        print(f"-> ERROR: Could not load checkpoint from {checkpoint_dir}. Reason: {e}")
        return None


In [4]:
# state_dict = load_and_prep_checkpoint("/scratch.global/lee02328/dynadiff/training_checkpoints/last.ckpt/")

In [5]:
# new_dict = {x: state_dict[x] for x in state_dict if "lora" not in x}
# new_dict = {x: new_dict[x] for x in new_dict if "unet" not in x}
# new_dict = {x: new_dict[x] for x in new_dict if "vae" not in x}

In [6]:
# new_dict.keys()

In [7]:
# new_dict['brain_modules.clip_image.out_time_agg.weight']
# new_dict['brain_modules.clip_image.subject_layers.weights'].shape

In [8]:
# import matplotlib.pyplot as plt
# import numpy as np
# import matplotlib.cm as cm # Import the colormap module

# fig, axes = plt.subplots(2, 2, figsize=(15, 12))
# axes = axes.ravel()

# cmap = cm.get_cmap('viridis').copy()

# cmap.set_bad(color='magenta')

# for i in range(4):
#     ax = axes[i]
    
#     weights_2d = new_dict['brain_modules.clip_image.subject_layers.weights'][i]
#     weights_np = weights_2d.detach().cpu().numpy()
    
#     masked_weights = np.ma.masked_where(np.abs(weights_np) < 1e-2, weights_np)

#     im = ax.imshow(masked_weights, aspect='auto', cmap=cmap)
    
#     fig.colorbar(im, ax=ax, label='Weight Value')
    
#     ax.set_title(f'Heatmap of Layer {i+1}')
#     ax.set_xlabel('Output Features (1552)')
#     ax.set_ylabel('Input Features (16724)')

# plt.tight_layout()
# plt.show()

In [9]:
import torch
from collections import OrderedDict, defaultdict

def merge_and_pad_weights(
    model_paths: list, 
    special_key: str, 
    special_key_padding_dim: int | None = None
):
    """
    Merges model weights sequentially to minimize memory usage, with an
    added capability to pad a special weight tensor to a uniform dimension.
    """
    if not isinstance(model_paths, list) or len(model_paths) < 2:
        raise ValueError("Please provide a list of at least two model file paths.")

    print(f"--- Starting weight stitching for {len(model_paths)} models (with padding) ---")

    summed_state_dict = {}
    tensor_counts = defaultdict(int)
    special_key_tensors = []

    for i, path in enumerate(model_paths):
        print(f"Processing model {i+1}/{len(model_paths)}: {path}")
        
        state_dict = load_and_prep_checkpoint(path)

        # --- Handle the special key with padding ---
        if special_key in state_dict:
            tensor = state_dict.pop(special_key)
            
            # Check if padding is needed for this special tensor
            if special_key_padding_dim is not None and tensor.shape[0] < special_key_padding_dim:
                current_features = tensor.shape[1]
                padding_needed = special_key_padding_dim - current_features
                
                # Pad format for 2D tensor (D1, D0): (pad_left_D0, pad_right_D0, pad_top_D1, pad_bottom_D1)
                # We only pad the bottom of the first dimension (features).
                pad_dims = (0, 0, 0, padding_needed) 
                padded_tensor = F.pad(tensor, pad_dims, "constant", 0)
                
                print(f"  -> PADDED Special Key: Original shape {list(tensor.shape)} -> Padded shape {list(padded_tensor.shape)}")
                special_key_tensors.append(padded_tensor)
            else:
                # No padding needed or specified
                special_key_tensors.append(tensor)
        
        # Accumulate all other weights
        for key, tensor in state_dict.items():
            if key not in summed_state_dict:
                summed_state_dict[key] = tensor.clone().to(torch.float32)
            elif summed_state_dict[key].shape == tensor.shape:
                summed_state_dict[key] += tensor.to(torch.float32)
            else:
                print(f"  -> WARNING: Shape mismatch for key '{key}'. "
                      f"Expected {summed_state_dict[key].shape}, got {tensor.shape}. Skipping.")
                continue
            
            tensor_counts[key] += 1

        del state_dict

    # --- Finalizing the merged dictionary ---
    merged_state_dict = OrderedDict()
    print("\n--- Averaging & Finalizing... ---")

    # Rule 1: Average all summed keys
    for key, summed_tensor in summed_state_dict.items():
        count = tensor_counts[key]
        if count > 0:
            merged_state_dict[key] = summed_tensor / count
            print(f"  -> Rule: AVERAGE. Averaged {count} tensors for key '{key}'")

    # Rule 2: Concatenate the (now padded) special key
    if special_key_tensors:
        if len(special_key_tensors) == len(model_paths):
            merged_tensor = torch.cat(special_key_tensors, dim=0)
            merged_state_dict[special_key] = merged_tensor
            shapes = ' + '.join([str(list(t.shape)) for t in special_key_tensors])
            print(f"  -> Rule: CONCAT. Shapes for '{special_key}': {shapes} -> {list(merged_tensor.shape)}")
        else:
            print(f"  -> WARNING: Special key '{special_key}' was not in all models. Concatenation skipped.")

    final_sorted_dict = OrderedDict(sorted(merged_state_dict.items()))
    
    print("\n--- Stitching complete. ---")
    return final_sorted_dict

In [10]:
BASE_PATH = "/scratch.global/lee02328/dynadiff/checkpoints"
for i in [7]:
    ## --- 1. DEFINE CONSTANTS & FILE PATHS --- ##
    SPECIAL_KEY = 'brain_modules.clip_image.subject_layers.weights'
    MAX_FMRI_FEATURES = 15724  # The padding dimension from your data prep
    OUTPUT_FILE = f"dynadiff_padded_sub{i:02d}.pth"

    # List of model file paths
    base_checkpoint_dir = [f"{BASE_PATH}/sub{i:02d}" for j in range(4)]


    ## --- 2. MERGE, PAD, AND SAVE THE WEIGHTS --- ##

    # Call the function with the new padding argument
    merged_weights = merge_and_pad_weights(
        base_checkpoint_dir, 
        SPECIAL_KEY, 
        special_key_padding_dim=MAX_FMRI_FEATURES
    )

    try:
        torch.save(merged_weights, OUTPUT_FILE)
        print(f"\n✅ Successfully saved merged model to: {OUTPUT_FILE}")
    except Exception as e:
        print(f"\n❌ Error saving file: {e}")

--- Starting weight stitching for 4 models (with padding) ---
Processing model 1/4: /scratch.global/lee02328/dynadiff/checkpoints/sub07
Loading checkpoint from: /scratch.global/lee02328/dynadiff/checkpoints/sub07
Processing zero checkpoint '/scratch.global/lee02328/dynadiff/checkpoints/sub07/checkpoint'


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 24.50it/s]


Detected checkpoint of type zero stage 2, world_size: 8
Parsing checkpoint created by deepspeed==0.15.1
Reconstructed Frozen fp32 state dict with 1649 params 1236735995 elements
Reconstructed fp32 state dict with 265 params 341467687 elements
-> Successfully loaded 1615 layers.
  -> PADDED Special Key: Original shape [1, 12682, 1552] -> Padded shape [1, 15724, 1552]
Processing model 2/4: /scratch.global/lee02328/dynadiff/checkpoints/sub07
Loading checkpoint from: /scratch.global/lee02328/dynadiff/checkpoints/sub07
Processing zero checkpoint '/scratch.global/lee02328/dynadiff/checkpoints/sub07/checkpoint'


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 74.46it/s]

Detected checkpoint of type zero stage 2, world_size: 8


...

Detected checkpoint of type zero stage 2, world_size: 8


Parsing checkpoint created by deepspeed==0.15.1
Reconstructed Frozen fp32 state dict with 1649 params 1236735995 elements
Reconstructed fp32 state dict with 265 params 341467687 elements
-> Successfully loaded 1615 layers.
  -> PADDED Special Key: Original shape [1, 12682, 1552] -> Padded shape [1, 15724, 1552]
Processing model 4/4: /scratch.global/lee02328/dynadiff/checkpoints/sub07
Loading checkpoint from: /scratch.global/lee02328/dynadiff/checkpoints/sub07
Processing zero checkpoint '/scratch.global/lee02328/dynadiff/checkpoints/sub07/checkpoint'


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 46.56it/s]

Detected checkpoint of type zero stage 2, world_size: 8


Parsing checkpoint created by deepspeed==0.15.1
Reconstructed Frozen fp32 state dict with 1649 params 1236735995 elements
Reconstructed fp32 state dict with 265 params 341467687 elements
-> Successfully loaded 1615 layers.
  -> PADDED Special Key: Original shape [1, 12682, 1552] -> Padded shape [1, 15724, 1552]

--- Averaging & Finalizing... ---
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.conv_in.weight'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.conv_in.bias'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.time_embedding.linear_1.weight'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.time_embedding.linear_1.bias'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.time_embedding.linear_2.weight'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.time_embedding.linear_2.bias'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.down_blocks.0.attentions.0.transformers.0.norm.weight'
  -> Rule: AVERAGE. Averaged 4 tensors for key 'unet.down_blocks.0.atten